<a href="https://colab.research.google.com/github/ofonime41/Malaria-Detection-/blob/main/MALARIA_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Malaria Detection ** using TensorFlow.
Dataset is gotten from: https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

In [24]:
!pip install tensorflow

In [15]:
# !pip uninstall keras

In [16]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [18]:
# Get PATH OF THE SATA DIRECTORY
train_dir = '/content/drive/MyDrive/Colab Notebooks/cell_images/Train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/cell_images/Test'

In [19]:
# IMAGINE RESIZING
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 500 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [21]:
# Load EfficientNetB0 without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

In [22]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [23]:
# Create full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 172s 10s/step - accuracy: 0.5988 - loss: 1.0404 - val_accuracy: 0.9640 - val_loss: 0.1479
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 63s 4s/step - accuracy: 0.8855 - loss: 0.2853 - val_accuracy: 0.9640 - val_loss: 0.1252
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.8794 - loss: 0.2653 - val_accuracy: 0.9680 - val_loss: 0.1237
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.8996 - loss: 0.2461 - val_accuracy: 0.9600 - val_loss: 0.1243
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.9188 - loss: 0.1949 - val_accuracy: 0.9560 - val_loss: 0.1234
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.9247 - loss: 0.1662 - val_accuracy: 0.9520 - val_loss: 0.1452
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 63s 4s/step - accuracy: 0.8949 - loss: 0.2156 - val_accuracy: 0.9560 - val_loss: 0.1494
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 62s 4s/step - accuracy: 0.9150 - loss: 0.2088 - val_accuracy: 0.9520 - val_los

In [12]:
# Save trained model
model.save('efficientnetb0_model.keras')

In [25]:
# Downloading trained model file for use
from google.colab import files
files.download("efficientnetb0_model.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>